In [1]:
# dependencies
import os
import re

In [2]:
# clone repository
os.system('git clone https://github.com/markedjs/marked.git')
os.chdir('marked')
# checkout vulnerable branch
os.system('git checkout a9696e2')

0

In [3]:
js_files = []
for root, dirs, files in os.walk('.'):
    for file in files:
        if file.endswith('.js'):
            js_files.append(os.path.join(root, file))

print(f"Found {len(js_files)} JavaScript files:")
for js_file in js_files:
    print(js_file)

Found 31 JavaScript files:
./rollup.config.esm.js
./rollup.config.js
./build-docs.js
./marked.min.js
./api/dingus.js
./docs/demo/demo.js
./docs/demo/worker.js
./bin/marked.js
./lib/marked.umd.js
./lib/marked.esm.js
./src/TextRenderer.js
./src/rules.js
./src/marked.js
./src/defaults.js
./src/Renderer.js
./src/Slugger.js
./src/Lexer.js
./src/Tokenizer.js
./src/Parser.js
./src/helpers.js
./test/rules.js
./test/update-specs.js
./test/bench.js
./test/vuln-regex.js
./test/unit/Parser-spec.js
./test/unit/Lexer-spec.js
./test/unit/marked-spec.js
./test/helpers/load.js
./test/helpers/html-differ.js
./test/helpers/helpers.js
./test/specs/run-spec.js


In [4]:
regex_strings = set()

for js_file in js_files:
    try:
        with open(js_file, 'r', encoding='utf-8') as f:
            content = f.read()
            # Find strings that look like regular expressions (e.g., /.../)
            found_regex = re.findall(r"=\s*\"\/((?:[^\\\/\"]*\\.)*[^\\\/\"]*)\/[gimyus]*\"", content)
            for regex in found_regex:
                regex_strings.add(regex)
            found_regex = re.findall(r"=\s*'\/((?:[^\\\/']*\\.)*[^\\\/']*)\/[gimyus]*'", content)
            for regex in found_regex:
                regex_strings.add(regex)
            found_regex = re.findall(r"=\s*\/((?:[^\\\/]*\\.)*[^\\\/]*)\/[gimyus]*", content)
            for regex in found_regex:
                regex_strings.add(regex)
            found_regex = re.findall(r":\s*\/((?:[^\\\/]*\\.)*[^\\\/]*)\/[gimyus]*", content)
            for regex in found_regex:
                regex_strings.add(regex)
    except Exception as e:
        print(f"Could not read file {js_file}: {e}")

print(f"Found {len(regex_strings)} unique regular expression strings:")
for regex in regex_strings:
    print(regex)

Found 77 unique regular expression strings:

\\\*|\\_
^ 
^__(?=\S)([\s\S]*?\S)__(?!_)|^\*\*(?=\S)([\s\S]*?\S)\*\*(?!\*)
\s+[a-zA-Z:_][\w.:-]*(?:\s*=\s*"[^"]*"|\s*=\s*'[^']*'|\s*=\s*[^\s"'=<>`]+)?
*#__PURE__*
[a-zA-Z][a-zA-Z0-9+.-]{1,31}
^[^_*]*?\*\*[^_*]*?\_[^_*]*?(?=\*\*)|[punct*](\_+)(?=[\s]|$)|[^punct*_\s](\_+)(?=[punct*\s]|$)|[punct*\s](\_+)(?=[^punct*_\s])|[\s](\_+)(?=[punct*])|[punct*](\_+)(?=[punct*])
^(?:\*+(?:([punct_])|[^\s*]))|^_+(?:([punct*])|([^\s_]))
\n *\n *$
^__|\*\*
^\\([!"#$%&'()*+,\-.
^([^'"]*[^\s])\s+(['"])(.*)\2
^ {0,3}\[(label)\]: *\n? *<?([^\s>]+)>?(?:(?: +\n? *| *\n *)(title))? *(?:\n+|$)
[A-Za-z0-9._+-]+(@)[a-zA-Z0-9-_]+(?:\.[a-zA-Z0-9-_]*[a-zA-Z0-9])+(?![-_])
(?:\[(?:\\.|[^\[\]\\])*\]|\\.|`[^`]*`|[^\[\]\\`])*?
^([^\n]+(?:\n(?!hr|heading|lheading|blockquote|fences|list|html|table| +\n)[^\n]+)*)
^:([^\n]*)(?:\n|$)
^[^:]+:\/*[^
[^ ]
^((?:ftp|https?):\/\/|www\.)(?:[a-zA-Z0-9\-]+\.?)+[^\s<]*|^email
^:([^\n]*):(?:\n|$)
[a-zA-Z0-9.!#$%&'*+
__(?!_)
^<(scheme:[^\s\x00-

In [5]:
redos = r"(?:([^\\)\]\/+*}]|\\.)|\((?:\?:)?(?:(?:[^)\\]*\\.)*[^)\\]*\|)?([^\\)\]\/+*}]|\\.)(?:\|(?:[^)\\]*\\.)*[^)\\]*)?\))[+*](?:(?:[^\\)\]\/+*}]|\\.|\((?:\?:)?(?:(?:[^)\\]*\\.)*[^)\\]*\|)?(?:[^\\)\]\/+*}]|\\.)(?:\|(?:[^)\\]*\\.)*[^)\\]*)?\))\?)?(?:(?:\1|\2)|\((?:\?:)?(?:(?:[^)\\]*\\.)*[^)\\]*\|)?(?:\1|\2)(?:\|(?:[^)\\]*\\.)*[^)\\]*)?\))[+*]"

for regex in regex_strings:
  if re.search(redos, regex):
    print(f"The following regex is vulnerable to redos: {regex}")


The following regex is vulnerable to redos: ^ {0,3}\[(label)\]: *\n? *<?([^\s>]+)>?(?:(?: +\n? *| *\n *)(title))? *(?:\n+|$)
